In [1]:
#import libs
import time
import cv2                                   


from matplotlib import pyplot as plt  
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
from pynq import MMIO
import random
from pynq import allocate
import numpy as np
import socket, time

In [2]:
#define data containers
pic_in_arr = allocate(shape=(600,800,3),dtype=np.uint8)
bg_in_arr = allocate(shape=(600,800,3),dtype=np.uint8)
mask_in_arr = allocate(shape=(600,800,3),dtype=np.uint8)
out_arr = allocate(shape=(600,800,3),dtype=np.uint8)
np_out = np.zeros((600,800,3), dtype=np.uint8)

In [2]:
#import ips
overlay = Overlay("./design_1.bit")
ip = overlay.top_0
#dma0 = overlay.axi_dma_0
#dma1 = overlay.axi_dma_1
#dma2 = overlay.axi_dma_2

pic_dma = overlay.axi_dma_1
mask_dma = overlay.axi_dma_0
back_dma = overlay.axi_dma_2

In [4]:
#get background picture
bg_img = cv2.imread("background2.jpg")

In [5]:
# #define internet transport interface
# dest_ip = ('192.168.31.56', 13579)
# UDP_server = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# UDP_server.bind(('192.168.137.100', 23579))
# example_image_path = './test.jpg'
# example_save_path = './test_result.jpg'

dest_ip = ('192.168.31.56', 13579)
UDP_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
UDP_server.connect(dest_ip)
example_image_path = './test.jpg'
# example_save_path = './test_result.jpg'
example_save_path = './test_result.jpg'

In [6]:
# def send_image(image, target_ip):
#     data = cv2.imencode('.jpg', image)[1].tobytes()
#     len_data = len(data)
#     space = (len_data + 9999) // 10000
#     UDP_server.sendto(space.to_bytes(4, 'big'), target_ip)
#     for i in range(0, space):
#         start_idx = i * 10000
#         end_idx = min(len_data, (i+1) * 10000)
#         UDP_server.sendto(data[start_idx: end_idx], target_ip)
#     return 'Send Image OK'

def send_image(image):
    data = cv2.imencode('.jpg', image)[1].tobytes()
    len_data = len(data)
    space = len_data
    UDP_server.sendall(space.to_bytes(16, 'big'))
    UDP_server.sendall(data)
    return 'Send Image  OK'

# def recv_image(image_path):
#     space, addr = UDP_server.recvfrom(10000)
#     space = int.from_bytes(space, 'big')
#     with open(image_path, 'wb') as f:
#         for i in range(space):
#             data, addr_now = UDP_server.recvfrom(10000)
#             assert addr == addr_now
#             f.write(data)
#     return 'recv Image OK'
def recv_image(image_path):
    space = UDP_server.recv(16)
    space = int.from_bytes(space, 'big')
    #print('space {}'.format(space))
    with open(image_path, 'wb') as f:
        while space > 0:
            data = UDP_server.recv(1024)
            #print('data len {}'.format(len(data)))
            space -= len(data)
            f.write(data)
    return 'Recv Image {} OK'.format(image_path)

In [17]:
readcap = cv2.VideoCapture("testread.mp4")

In [18]:
cap2 = cv2.VideoCapture(1) 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('testwrite.mp4',fourcc, 10.0, (800,600),True)

In [19]:
while True:
    ret, np_frame = readcap.read()
    if not ret:
        break
    np_frame = cv2.resize(np_frame,(800,600))
    send_image(np_frame)
    recv_image(example_save_path)
    mask_img = cv2.imread("test_result.jpg")
    np.copyto(pic_in_arr,np_frame)
    np.copyto(bg_in_arr,bg_img)
    np.copyto(mask_in_arr,mask_img)
    #dma
    pic_dma.sendchannel.transfer(pic_in_arr)
    mask_dma.sendchannel.transfer(mask_in_arr)
    back_dma.sendchannel.transfer(bg_in_arr)
    mask_dma.recvchannel.transfer(out_arr)
    ip.write(0x00, 0x1)
    isready = ip.read(0x00)
    while (isready == 1):
        isready = ip.read(0x00)
    pic_dma.sendchannel.wait()
    mask_dma.sendchannel.wait()
    back_dma.sendchannel.wait()
    mask_dma.recvchannel.wait()
    np.copyto(np_out,out_arr)
    out.write(np_out)
    print("ok")

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [10]:
#import ips
overlay = Overlay("./design_1.bit")
ip = overlay.top_0
#dma0 = overlay.axi_dma_0
#dma1 = overlay.axi_dma_1
#dma2 = overlay.axi_dma_2

pic_dma = overlay.axi_dma_1
mask_dma = overlay.axi_dma_0
back_dma = overlay.axi_dma_2

In [20]:
cap2.release()
out.release()

In [16]:
readcap.release()

In [3]:
from pynq import Overlay,allocate
import pynq.lib.video
from pynq.lib.video.common import PixelFormat,VideoMode

framemode = VideoMode(800,600,32)
vdma=overlay.axi_vdma_0
vdma.writechannel.mode=framemode
vdma.writechannel.start()
vdma.writechannel.parked

#buf=np.zeros((800,600,3),dtype=np.int8)
buf=vdma.writechannel.newframe()
#buf=np.zeros((600,800,3),dtype=np.int32)

#buf[:,:,:]=np.zeros((600,800,4),dtype=np.uint8)


vdma.writechannel.setframe(buf)
tpg = overlay.v_tpg_0
tpg.write(0x10,600)
tpg.write(0x18,800)
tpg.write(0x40,0)
tpg.write(0x20,0x9)
tpg.write(0x0,0x81)

bin(tpg.read(0x0))
tpg.read(0x4)
tpg.read(0xc)

0

In [4]:
mix = overlay.v_mix_0
mix.write(0x4,0)
mix.write(0x10,800)
mix.write(0x18,600)

In [5]:
mix.write(0x100,0)
mix.write(0x108,0)
mix.write(0x110,0)
mix.write(0x118,800)
mix.write(0x120,48)
mix.write(0x128,600)
mix.write(0x130,0)

mix.write(0x200,255)
mix.write(0x208,0)
mix.write(0x210,0) 
mix.write(0x218,800)
mix.write(0x220,48)
mix.write(0x228,600)
mix.write(0x230,0)

In [6]:
mix.write(0x40,0b11)
mix.write(0x0,0x81)

In [29]:
readcap2 = cv2.VideoCapture("testwrite.mp4")

In [30]:
while True:
    ret, np_frame = readcap2.read()
    if not ret:
        break
    buf[:,:,0]=np_frame[:,:,1]
    buf[:,:,1]=np_frame[:,:,0]
    buf[:,:,2]=np_frame[:,:,2]
    buf[:,:,3]=0xff
    time.sleep(0.05)

In [28]:
readcap2.release()